In [ ]:
from utils import import_dataset_fct
import numpy as np
import pandas as pd
import random

random.seed(42)

In [ ]:
import_dataset_fct.return_filenames()

In [ ]:
df = import_dataset_fct.get_file("Dataset_1_train_asset.csv")
df

In [ ]:
def col_numeric_names(df):
    df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
    return df

def absolute_growth(df):
    len_df = df.shape[1]
    temp = df.iloc[:,1:len_df]
    temp = col_numeric_names(temp)
    temp2 = df.iloc[:,:len_df-1]
    df2 = temp-temp2
    df2.insert(0, "0", 0)
    df2 = col_numeric_names(df2)
    return df2
df_absolute_growth = absolute_growth(df)
df_absolute_growth

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:

data = df.iloc[:,:30]
payoff_data = pd.read_csv(url_payoff, header=None)

In [ ]:
def col_numeric_names(df):
    df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
    return df

def absolute_growth(df):
    len_df = df.shape[1]
    temp = df.iloc[:,1:len_df]
    temp = col_numeric_names(temp)
    temp2 = df.iloc[:,:len_df-1]
    df2 = temp-temp2
    df2.insert(0, "0", 0)
    df2 = col_numeric_names(df2)
    return df2

def get_train_val(url_data, url_payoff, split_percent=0.8):
    df = import_dataset_fct.get_file(url_data)
    pay_off = pd.read_csv(url_payoff, header=None)
    df_absolute_growth = absolute_growth(df)
    data = np.array(df.iloc[:,:30].values.astype('float32'))
    #scaler = MinMaxScaler(feature_range=(0, 1))
    #data = scaler.fit_transform(data).flatten()
    pay_off = np.array(pay_off)
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    val_data = data[split:]
    n = len(df)
    split = int(n*split_percent)
    train_payoff = pay_off[range(split)]
    train_growth = np.array(df_absolute_growth.iloc[0:split, 1:])
    val_payoff = pay_off[split:]
    val_growth = np.array(df_absolute_growth.iloc[split:, 1:])
    #train_y = np.append(train_payoff, train_growth, axis=1)
    #val_y = np.append(val_payoff, val_growth, axis=1)
    return train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, data

url_payoff = "/home/jovyan/hfactory_magic_folders/natixis_data_challenge_22_23/erm/Dataset_1_train_payoff.csv"
url_data = "Dataset_1_train_asset.csv"
train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, data = get_train_val(url_data, url_payoff, split_percent=0.8)

In [ ]:
#print(list(torch.utils.data.TensorDataset(torch.Tensor(train_growth), torch.Tensor(train_payoff))))

In [ ]:
def prepare_data(train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, batch_size):
    train_payoff = torch.Tensor(train_payoff)
    val_payoff = torch.Tensor(val_payoff)
    train_growth = torch.Tensor(train_growth)
    val_growth = torch.Tensor(val_growth)
    train_X = torch.Tensor(train_data)
    val_X = torch.Tensor(val_data)
    dataset_train = torch.utils.data.TensorDataset(train_X, train_payoff, train_growth)
    dataset_val = torch.utils.data.TensorDataset(val_X, val_payoff, val_growth)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
    return dataset_train, dataset_val, train_loader, val_loader

batch_size = 32
dataset_train, dataset_val, train_loader, val_loader = prepare_data(train_data, val_data, train_payoff, val_payoff, train_growth, val_growth, batch_size)

In [ ]:
class DeepHedging_RNN(nn.Module):
    def __init__(self, input_size, HL_size, output_size):
        super(DeepHedging_RNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size=input_size,
                                hidden_size=HL_size,
                                num_layers=1)
        self.linear = torch.nn.Linear(HL_size, output_size)
    def forward(self, S):
        out, _ = self.rnn(S)
        out = self.linear(out)
        return out.squeeze()

In [ ]:
torch.Tensor(train_growth)

In [ ]:
torch.transpose(torch.Tensor(train_growth), 0, 1)

In [ ]:
train_growth.shape

In [ ]:
def loss(deltas, pay_off, growth):
    S_delta_T = torch.transpose(growth, 0, 1)
    delta_S = torch.diagonal(torch.matmul(deltas, S_delta_T))
    loss = torch.sub(pay_off, delta_S)
    worst_one_perc = torch.topk(-loss, 3)
    es_99 = torch.mean(- torch.topk(-loss, 3)[0])
    es_50 = torch.mean(- torch.topk(-loss, 15)[0])
    risk_measure = (es_50 + es_99)/2
    return risk_measure

In [ ]:
train_data.size

In [ ]:
num_epochs = 50
learning_rate = 0.1

input_size = 1 # number of expected features
output_size = 30
HL_size = 3
model = DeepHedging_RNN(input_size, HL_size, output_size)
# Set up optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from tqdm import tqdm

train_loss = []
val_loss = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    counter = 0

    for train_X, train_payoff, train_growth in train_loader:
        counter += 1

        # Calculate gradients and update model weights
        optimizer.zero_grad()
        train_X = train_X.unsqueeze(-1)
        deltas = model(train_X)
        losses = loss(deltas, train_payoff, train_growth)
        running_loss += losses.item()
        
        losses.backward()
        optimizer.step()
        
    epoch_train_loss = running_loss / counter
    train_loss.append(epoch_train_loss)
    print("Train loss: {}".format(epoch_train_loss))
    
    running_loss_test = 0.0
    counter = 0

    for val_X, val_payoff, val_growth in val_loader:
        model.eval()
        
        counter += 1
        val_X = val_X.unsqueeze(-1)
        deltas = model(val_X)
        losses_test = loss(deltas, val_payoff, val_growth)
        running_loss_test += losses_test.item()

    epoch_test_loss = running_loss_test / counter
    val_loss.append(epoch_test_loss)
    print("Test loss: {}".format(epoch_test_loss))